In [ ]:
p1 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained_reprojected/scans/scene0000_00/create_labels_from_pretrained_reprojected"
p2 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained/scans/scene0000_00/create_labels_from_pretrained"
p3 = "/home/jonfrey/Datasets/labels_generated/create_labels_from_gt/scans/scene0000_00/create_labels_from_gt"

from pathlib import Path

p1 = [str(s) for s in Path(p1).rglob('*.png')]
p1.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)
p2 = [str(s) for s in Path(p2).rglob('*.png')]
p2.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)
p3 = [str(s) for s in Path(p3).rglob('*.png')]
p3.sort(
    key=lambda x: int(x.split("/")[-1][:-4])
)

images = [ s.replace("/home/jonfrey/Datasets/labels_generated/create_labels_from_pretrained_reprojected/scans/scene0000_00/create_labels_from_pretrained_reprojected", "/home/jonfrey/Datasets/scannet/scans/scene0000_00/color").replace('png', 'jpg') for s in p1 ]
import sys
import os
os.chdir("/home/jonfrey/ASL")
sys.path.append("""/home/jonfrey/ASL/src/""")
sys.path.append("""/home/jonfrey/ASL/src/pseudo_label""")
from utils_asl import LabelLoaderAuto
from visu import Visualizer
from PIL import Image
visu = Visualizer("/home/jonfrey/Documents/master_thesis/Report/images/reprojected_pretrained", 
                  logger=None, 
                  epoch=0, store=True, num_classes=41)

lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

In [ ]:
replot_sequence = False
if replot_sequence:
    for k in range(len(p1)):
        l = lla.get(p1[k])[0]
        img = np.array( Image.open( images[k]) )
        visu.plot_detectron(img, l, alpha=0.8 ,tag="reprojected", jupyter=False, text_off=True,epoch=k*10)


In [ ]:
import copy
import numpy as np

class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
        
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s
        
accm = AccMonitor()
accm.update("Test", np.ones( (100,100)) ,np.ones( (100,100)))
accm.update("Test2", np.ones( (100,100)) ,np.ones( (100,100)))
print(accm)

In [ ]:
p3

In [ ]:
plot_in_jupyter = False
accm = AccMonitor()
for k in range(len(p1)):
    gt = lla.get(p3[k*10])[0]
    network =  lla.get(p2[k])[0]
    reprojected = lla.get(p1[k])[0]
    if plot_in_jupyter:
        visu.plot_detectron(img, gt, alpha=0.8 ,store=False,jupyter=True)
        visu.plot_detectron(img, network, alpha=0.8 ,store=False,jupyter=True)
        visu.plot_detectron(img, reprojected, alpha=0.8 ,store=False,jupyter=True)
    accm.update("Reprojected", reprojected-1 , gt-1)
    accm.update("Network", network-1,gt-1)
    
    print( accm )

In [ ]:
me = accm.metrics

s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s